# Fantasy Premier League Data Audit Notebook

This notebook performs comprehensive data quality assessment across all seasons,
validating schema consistency, identifying missing values, and examining temporal
coverage.


## CONFIGURATION

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

BASE_DIR = Path.cwd().parent if 'Notebooks' in str(Path.cwd()) else Path.cwd()
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"

HISTORICAL_SEASONS = ["2021-22", "2022-23", "2023-24", "2024-25"]
CURRENT_SEASON = "2025-26"

print(f"Base Directory: {BASE_DIR}")
print(f"Raw Data Directory: {RAW_DIR}")
print(f"Processed Data Directory: {PROCESSED_DIR}")
print("\n" + "="*80 + "\n")

Base Directory: c:\Users\kadmi\Workspace\02_Projects\fpl-advisor-engine
Raw Data Directory: c:\Users\kadmi\Workspace\02_Projects\fpl-advisor-engine\data\raw
Processed Data Directory: c:\Users\kadmi\Workspace\02_Projects\fpl-advisor-engine\data\processed




## SECTION 1: FILE INVENTORY

In [2]:
available_files = {}

for season in HISTORICAL_SEASONS + [CURRENT_SEASON]:
    merged_file = RAW_DIR / f"{season}_merged_gw.csv"
    teams_file = RAW_DIR / f"{season}_teams.csv"
    
    files_found = {
        'merged_gw': merged_file.exists(),
        'teams': teams_file.exists()
    }
    
    available_files[season] = files_found
    
    status = "COMPLETE" if all(files_found.values()) else "INCOMPLETE"
    print(f"{season}: {status}")
    if not files_found['merged_gw']:
        print(f"  Missing: {season}_merged_gw.csv")
    if not files_found['teams']:
        print(f"  Missing: {season}_teams.csv")

print("\n" + "="*80 + "\n")

2021-22: COMPLETE
2022-23: COMPLETE
2023-24: COMPLETE
2024-25: COMPLETE
2025-26: COMPLETE




## SECTION 2: SCHEMA ANALYSIS

In [3]:
schema_info = {}

for season in HISTORICAL_SEASONS + [CURRENT_SEASON]:
    merged_file = RAW_DIR / f"{season}_merged_gw.csv"
    
    if not merged_file.exists():
        print(f"{season}: File not found, skipping...")
        continue
    
    try:
        df = pd.read_csv(merged_file, nrows=5)
        schema_info[season] = {
            'columns': set(df.columns),
            'n_columns': len(df.columns),
            'dtypes': df.dtypes.to_dict()
        }
        print(f"{season}: {len(df.columns)} columns detected")
    except Exception as e:
        print(f"{season}: Error reading file - {str(e)}")
        schema_info[season] = None
# Identify common columns across all seasons
if schema_info:
    all_columns = [info['columns'] for info in schema_info.values() if info is not None]
    
    if all_columns:
        common_columns = set.intersection(*all_columns)
        all_unique_columns = set.union(*all_columns)
        
        print(f"\nCommon columns across all seasons: {len(common_columns)}")
        print(f"Total unique columns across all seasons: {len(all_unique_columns)}")
        
        # Identify columns that exist in some seasons but not others
        inconsistent_columns = all_unique_columns - common_columns
        if inconsistent_columns:
            print("\nInconsistent columns (not present in all seasons):")
            for col in sorted(inconsistent_columns):
                seasons_with_col = [s for s, info in schema_info.items() 
                                   if info and col in info['columns']]
                print(f"  - {col}: present in {seasons_with_col}")

print("\n" + "="*80 + "\n")

2021-22: 37 columns detected
2022-23: 42 columns detected
2023-24: 42 columns detected
2024-25: 43 columns detected
2025-26: 43 columns detected

Common columns across all seasons: 33
Total unique columns across all seasons: 47

Inconsistent columns (not present in all seasons):
  - GW: present in ['2021-22', '2022-23', '2023-24', '2024-25']
  - clearances_blocks_interceptions: present in ['2025-26']
  - defensive_contribution: present in ['2025-26']
  - expected_assists: present in ['2022-23', '2023-24', '2024-25', '2025-26']
  - expected_goal_involvements: present in ['2022-23', '2023-24', '2024-25', '2025-26']
  - expected_goals: present in ['2022-23', '2023-24', '2024-25', '2025-26']
  - expected_goals_conceded: present in ['2022-23', '2023-24', '2024-25', '2025-26']
  - modified: present in ['2024-25', '2025-26']
  - position: present in ['2021-22', '2022-23', '2023-24', '2024-25']
  - recoveries: present in ['2025-26']
  - starts: present in ['2022-23', '2023-24', '2024-25', '202

## SECTION 3: DETAILED DATA PROFILING (First season)

In [4]:
# Load first available season for detailed inspection
first_season = None
for season in HISTORICAL_SEASONS:
    merged_file = RAW_DIR / f"{season}_merged_gw.csv"
    if merged_file.exists():
        first_season = season
        break

if first_season:
    print(f"Analyzing {first_season} as reference season...\n")
    
    df_sample = pd.read_csv(RAW_DIR / f"{first_season}_merged_gw.csv")
    
    print(f"Dataset Shape: {df_sample.shape[0]:,} rows × {df_sample.shape[1]} columns")
    print(f"Memory Usage: {df_sample.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print("\nColumn Inventory:")
    print("-" * 80)
    for i, col in enumerate(df_sample.columns, 1):
        dtype = df_sample[col].dtype
        n_missing = df_sample[col].isna().sum()
        pct_missing = (n_missing / len(df_sample)) * 100
        n_unique = df_sample[col].nunique()
        
        print(f"{i:2d}. {col:25s} | {str(dtype):10s} | "
              f"Missing: {n_missing:6d} ({pct_missing:5.2f}%) | "
              f"Unique: {n_unique:6d}")
    
    print("\nTarget Variable Statistics (total_points):")
    print("-" * 80)
    if 'total_points' in df_sample.columns:
        points_stats = df_sample['total_points'].describe()
        print(points_stats)
        
        print(f"\nZero-point matches: {(df_sample['total_points'] == 0).sum():,} "
              f"({(df_sample['total_points'] == 0).sum() / len(df_sample) * 100:.1f}%)")
    
    print("\nGameweek Coverage:")
    print("-" * 80)
    if 'GW' in df_sample.columns:
        gw_coverage = df_sample['GW'].value_counts().sort_index()
        print(f"Gameweeks present: {gw_coverage.index.min()} to {gw_coverage.index.max()}")
        print(f"Total gameweeks: {len(gw_coverage)}")
        print("\nPlayers per gameweek (sample):")
        print(gw_coverage.head(10))
    elif 'round' in df_sample.columns:
        round_coverage = df_sample['round'].value_counts().sort_index()
        print(f"Rounds present: {round_coverage.index.min()} to {round_coverage.index.max()}")
        print(f"Total rounds: {len(round_coverage)}")
    
    print("\nPosition Distribution:")
    print("-" * 80)
    if 'position' in df_sample.columns:
        pos_dist = df_sample['position'].value_counts()
        print(pos_dist)
    elif 'element_type' in df_sample.columns:
        element_dist = df_sample['element_type'].value_counts()
        print("Element Type Distribution (1=GK, 2=DEF, 3=MID, 4=FWD):")
        print(element_dist)
    
    print("\nKey Performance Metrics - Sample Statistics:")
    print("-" * 80)
    key_metrics = ['total_points', 'minutes', 'goals_scored', 'assists', 
                   'clean_sheets', 'ict_index', 'influence', 'creativity', 'threat']
    
    available_metrics = [m for m in key_metrics if m in df_sample.columns]
    
    if available_metrics:
        summary = df_sample[available_metrics].describe().T
        summary['missing'] = df_sample[available_metrics].isna().sum()
        summary['missing_pct'] = (summary['missing'] / len(df_sample)) * 100
        print(summary[['count', 'mean', 'std', 'min', 'max', 'missing', 'missing_pct']])

else:
    print("No historical season data found for detailed profiling.")

print("\n" + "="*80 + "\n")

Analyzing 2021-22 as reference season...

Dataset Shape: 25,447 rows × 37 columns
Memory Usage: 13.32 MB

Column Inventory:
--------------------------------------------------------------------------------
 1. name                      | object     | Missing:      0 ( 0.00%) | Unique:    735
 2. position                  | object     | Missing:      0 ( 0.00%) | Unique:      5
 3. team                      | object     | Missing:      0 ( 0.00%) | Unique:     20
 4. xP                        | float64    | Missing:      0 ( 0.00%) | Unique:    174
 5. assists                   | int64      | Missing:      0 ( 0.00%) | Unique:      5
 6. bonus                     | int64      | Missing:      0 ( 0.00%) | Unique:      4
 7. bps                       | int64      | Missing:      0 ( 0.00%) | Unique:     87
 8. clean_sheets              | int64      | Missing:      0 ( 0.00%) | Unique:      2
 9. creativity                | float64    | Missing:      0 ( 0.00%) | Unique:    634
10. element 

## SECTION 4: TEMPORAL COVERAGE SUMMARY

In [5]:
total_gameweeks = 0
total_observations = 0

for season in HISTORICAL_SEASONS + [CURRENT_SEASON]:
    merged_file = RAW_DIR / f"{season}_merged_gw.csv"
    
    if not merged_file.exists():
        continue
    
    try:
        df_temp = pd.read_csv(merged_file)
        
        # Determine gameweek column
        gw_col = 'GW' if 'GW' in df_temp.columns else 'round'
        
        if gw_col in df_temp.columns:
            n_gw = df_temp[gw_col].nunique()
            gw_range = f"{df_temp[gw_col].min()}-{df_temp[gw_col].max()}"
        else:
            n_gw = "Unknown"
            gw_range = "N/A"
        
        n_obs = len(df_temp)
        
        print(f"{season}: {n_obs:6,} observations across {n_gw} gameweeks (Range: {gw_range})")
        
        if isinstance(n_gw, int):
            total_gameweeks += n_gw
        total_observations += n_obs
        
    except Exception as e:
        print(f"{season}: Error - {str(e)}")

print(f"\nTotal Temporal Coverage: {total_gameweeks} gameweeks")
print(f"Total Observations: {total_observations:,}")

print("\n" + "="*80 + "\n")

2021-22: 25,447 observations across 38 gameweeks (Range: 1-38)
2022-23: 26,505 observations across 37 gameweeks (Range: 1-38)
2023-24: 29,725 observations across 38 gameweeks (Range: 1-38)
2024-25: 14,178 observations across 21 gameweeks (Range: 1-21)
2025-26: 10,331 observations across 14 gameweeks (Range: 1-14)

Total Temporal Coverage: 148 gameweeks
Total Observations: 106,186




## SECTION 5: DATA QUALITY ISSUES

In [6]:
if first_season:
    df_quality = pd.read_csv(RAW_DIR / f"{first_season}_merged_gw.csv")
    
    print("Critical Fields Missing Value Analysis:")
    print("-" * 80)
    
    critical_fields = ['total_points', 'minutes', 'element', 'opponent_team', 'was_home']
    
    for field in critical_fields:
        if field in df_quality.columns:
            n_missing = df_quality[field].isna().sum()
            pct_missing = (n_missing / len(df_quality)) * 100
            status = "⚠ WARNING" if pct_missing > 5 else "✓ OK"
            print(f"{field:20s}: {n_missing:6d} missing ({pct_missing:5.2f}%) {status}")
    
    print("\nDuplicate Records Check:")
    print("-" * 80)
    
    if 'element' in df_quality.columns and 'GW' in df_quality.columns:
        n_duplicates = df_quality.duplicated(subset=['element', 'GW']).sum()
        print(f"Duplicate player-gameweek combinations: {n_duplicates}")
    elif 'element' in df_quality.columns and 'round' in df_quality.columns:
        n_duplicates = df_quality.duplicated(subset=['element', 'round']).sum()
        print(f"Duplicate player-round combinations: {n_duplicates}")
    
    print("\nOutlier Detection (Total Points):")
    print("-" * 80)
    if 'total_points' in df_quality.columns:
        q99 = df_quality['total_points'].quantile(0.99)
        n_extreme = (df_quality['total_points'] > q99).sum()
        print(f"99th percentile: {q99:.1f} points")
        print(f"Observations above 99th percentile: {n_extreme} ({n_extreme/len(df_quality)*100:.2f}%)")
        
        if n_extreme > 0:
            print("\nTop 10 extreme performances:")
            extreme_cols = ['name', 'position', 'total_points', 'minutes', 'goals_scored', 'assists']
            available_extreme = [c for c in extreme_cols if c in df_quality.columns]
            if available_extreme:
                print(df_quality.nlargest(10, 'total_points')[available_extreme].to_string(index=False))

print("\n" + "="*80 + "\n")
print("DATA AUDIT COMPLETE")
print("="*80)

Critical Fields Missing Value Analysis:
--------------------------------------------------------------------------------
total_points        :      0 missing ( 0.00%) ✓ OK
minutes             :      0 missing ( 0.00%) ✓ OK
element             :      0 missing ( 0.00%) ✓ OK
opponent_team       :      0 missing ( 0.00%) ✓ OK
was_home            :      0 missing ( 0.00%) ✓ OK

Duplicate Records Check:
--------------------------------------------------------------------------------
Duplicate player-gameweek combinations: 2217

Outlier Detection (Total Points):
--------------------------------------------------------------------------------
99th percentile: 12.0 points
Observations above 99th percentile: 190 (0.75%)

Top 10 extreme performances:
                         name position  total_points  minutes  goals_scored  assists
                  Mason Mount      MID            24       90             3        1
                Mohamed Salah      MID            24       90             3    